In [63]:
import langchain
from langchain_community.document_loaders import ToMarkdownLoader, ReadTheDocsLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter, MarkdownTextSplitter
from langchain.schema import Document
import re
from pathlib import Path

In [159]:
def preprocess_markdown(md_text: str):
    """
    Làm s?ch và chu?n hóa n?i dung Markdown:
    - Gi? nguyên tiêu d?, b?ng, code block
    - Lo?i b? hình ?nh và ph?n tham kh?o
    """
    # 1?? Lo?i b? hình ?nh ![...](...)
    md_text = re.sub(r'!\[.*?\]\(.*?\)', '', md_text)

    # 2?? Lo?i b? liên k?t tr?n [text](url) ? gi? l?i text
    md_text = re.sub(r'\[(.*?)\]\(.*?\)', r'\1', md_text)

    # 3?? Lo?i b? ph?n References (n?u có)
    md_text = re.split(r'(?i)^#+\s*references', md_text)[0]

    # 4?? Chu?n hóa nhi?u dòng tr?ng
    md_text = re.sub(r'\n{3,}', '\n\n', md_text)

    return md_text.strip()


def split_markdown_into_sections(md_text: str):
    """
        split document according to header using MardownHeaderTextSplitter
    """
    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]
    markdown_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=headers_to_split_on,
        return_each_line=False,
        strip_headers=False
    )
    markdown_splits = markdown_splitter.split_text(md_text)
    return markdown_splits


def create_chunks(md_text: str, source_path: str, chunk_size: int=512, chunk_overlap=30):
    """
        create chunks by apply recursive splitter method within markdown_split
    """
    # md_text = preprocess_markdown(md_text)
    markdown_splits = split_markdown_into_sections(md_text)
    # text_splitter = RecursiveCharacterTextSplitter(
    #     separators=[",", ","],
    #     chunk_size=chunk_size,
    #     chunk_overlap=chunk_overlap
    # )
    # splits = text_splitter.split_documents(markdown_splits)

    return markdown_splits

    

# ?? Ví d? s? d?ng
if __name__ == "__main__":
    path = "/data/AIRACE/training_out/Public003/main.md"
    md_text = Path(path).read_text(encoding="utf-8")

    md_splits = create_chunks(md_text, source_path=path)
    print(len(md_splits))
    

13


In [9]:
from chonkie import Pipeline
from chonkie import RecursiveRules, RecursiveLevel
from chonkie import TokenChunker, OverlapRefinery


rules = RecursiveRules(
    levels=[
        RecursiveLevel(delimiters=["\n\n"], include_delim="prev"),
        RecursiveLevel(delimiters=["\n"], include_delim="prev"),
        RecursiveLevel(delimiters=["."], include_delim="prev"),
        RecursiveLevel(delimiters=[","], include_delim="prev"),
        RecursiveLevel(whitespace=False)
    ]
)
# Build and execute pipeline
doc = (Pipeline()
    .fetch_from("file", path="/data/AIRACE/training_out/Public003/main.md")
    .process_with("markdown")
    .chunk_with("recursive", chunk_size=512)
    .refine_with("overlap", context_size=50, mode="recursive", rules=rules)
    .run())

# Access chunks
# print(f"Created {len(doc.chunks)} chunks")
# for i, chunk in enumerate(doc.chunks):
#     print(f"=========== chunk {i} =============")
#     print(chunk.text)

# print(f"Found {len(doc.tables)} tables")
# print(f"Found {len(doc.code)} code blocks")
# print(f"Found {len(doc.images)} images")
# print(f"Created {len(doc.chunks)} chunks")
# print(type(doc))


overlap_refinery = OverlapRefinery(
    tokenizer="character",
    context_size=0.5,
    method="suffix",
    merge=True
)



In [13]:
from chonkie import MarkdownChef
# Initialize the chef
chef = MarkdownChef()
doc = chef.process("/data/AIRACE/training_out/Public003/main.md")
print(doc.chunks)


[Chunk(text='# Public_003

_Dịch máy là một trong những hướng nghiên cứu quan trọng trong xử lý ngôn ngữ tự nhiên. Trong những năm gần đây, dịch máy nơ ron đã và đang được nghiên cứu phổ biến hơn trong cộng đồng dịch máy vì hiện tại nó cho chất lượng dịch tốt hơn so với phương pháp dịch máy thống kê truyền thống. Tuy nhiên, dịch máy nơ ron lại cần lượng lớn dữ liệu song ngữ để huấn luyện. Hệ dịch sẽ cho chất lượng bản dịch tốt hơn khi nó được thử nghiệm trong cùng miền với miền dữ liệu mà nó được huấn luyện, ngược lại thì chất lượng bản dịch sẽ bị sụt giảm, mức độ sụt giảm phụ thuộc vào mức độ khác biệt giữa dữ liệu miền huấn luyện và dữ liệu miền thử nghiệm. Hiện nay, các kĩ thuật thích ứng miền cho dịch máy nơ ron đã được công bố chủ yếu được thực hiện trên một số cặp ngôn ngữ phổ biến giàu tài nguyên, và chưa có nhiều nghiên cứu đã được công bố về thích ứng miền trong dịch máy nơ ron cho cặp ngôn ngữ Anh - Việt._

_Trong bài báo này, chúng tôi đề xuất một phương pháp thích ứng miền 

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-4B-Instruct-2507", cache_dir="/data/AIRACE/RAG")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-4B-Instruct-2507", cache_dir= "/data/AIRACE/RAG")
messages = [
    {"role": "user", "content": "Who are you?"},
]

/opt/miniconda3/envs/agent/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/miniconda3/envs/agent/lib/python3.10/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
Loading checkpoint shards: 100%|██████████| 3/3 [01:55<00:00, 38.35s/it]


In [19]:
tables = '''| Model                                                                                                                                                                    | Size<br><sup>(pixels) | mAP<sup>val<br>50-95 | mAP<sup>val<br>50 | Speed<br><sup>CPU b1<br>(ms) | Speed<br><sup>V100 b1<br>(ms) | Speed<br><sup>V100 b32<br>(ms) | Params<br><sup>(M) | FLOPs<br><sup>@640 (B) |
| ------------------------------------------------------------------------------------------------------------------------------------------------------------------------ | --------------------- | -------------------- | ----------------- | ---------------------------- | ----------------------------- | ------------------------------ | ------------------ | ---------------------- |
| [YOLOv5n](https://github.com/ultralytics/yolov5/releases/download/v7.0/yolov5n.pt)                                                                                       | 640                   | 28.0                 | 45.7              | **45**                       | **6.3**                       | **0.6**                        | **1.9**            | **4.5**                |
| [YOLOv5s](https://github.com/ultralytics/yolov5/releases/download/v7.0/yolov5s.pt)                                                                                       | 640                   | 37.4                 | 56.8              | 98                           | 6.4                           | 0.9                            | 7.2                | 16.5                   |
| [YOLOv5m](https://github.com/ultralytics/yolov5/releases/download/v7.0/yolov5m.pt)                                                                                       | 640                   | 45.4                 | 64.1              | 224                          | 8.2                           | 1.7                            | 21.2               | 49.0                   |
| [YOLOv5l](https://github.com/ultralytics/yolov5/releases/download/v7.0/yolov5l.pt)                                                                                       | 640                   | 49.0                 | 67.3              | 430                          | 10.1                          | 2.7                            | 46.5               | 109.1                  |
| [YOLOv5x](https://github.com/ultralytics/yolov5/releases/download/v7.0/yolov5x.pt)                                                                                       | 640                   | 50.7                 | 68.9              | 766                          | 12.1                          | 4.8                            | 86.7               | 205.7                  |
|                                                                                                                                                                          |                       |                      |                   |                              |                               |                                |                    |                        |
| [YOLOv5n6](https://github.com/ultralytics/yolov5/releases/download/v7.0/yolov5n6.pt)                                                                                     | 1280                  | 36.0                 | 54.4              | 153                          | 8.1                           | 2.1                            | 3.2                | 4.6                    |
| [YOLOv5s6](https://github.com/ultralytics/yolov5/releases/download/v7.0/yolov5s6.pt)                                                                                     | 1280                  | 44.8                 | 63.7              | 385                          | 8.2                           | 3.6                            | 12.6               | 16.8                   |
| [YOLOv5m6](https://github.com/ultralytics/yolov5/releases/download/v7.0/yolov5m6.pt)                                                                                     | 1280                  | 51.3                 | 69.3              | 887                          | 11.1                          | 6.8                            | 35.7               | 50.0                   |
| [YOLOv5l6](https://github.com/ultralytics/yolov5/releases/download/v7.0/yolov5l6.pt)                                                                                     | 1280                  | 53.7                 | 71.3              | 1784                         | 15.8                          | 10.5                           | 76.8               | 111.4                  |
| [YOLOv5x6](https://github.com/ultralytics/yolov5/releases/download/v7.0/yolov5x6.pt)<br>+ [[TTA]](https://docs.ultralytics.com/yolov5/tutorials/test_time_augmentation/) | 1280<br>1536          | 55.0<br>**55.8**     | 72.7<br>**72.7**  | 3136<br>-                    | 26.2<br>-                     | 19.4<br>-                      | 140.7<br>-         | 209.8<br>-             |'''

In [2]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
messages = [
    {"role": "system", "content": "Bạn là chuyên gia ngôn ngữ có khả năng tổng hợp thông tin từ bảng biểu."},
    {"role": "user", "content": f"Hãy viết một đoạn mô tả bảng sau {tables} đảm bảo các yếu tố sau:\n 1) Câu trả lời bằng tiếng Việt.\n 2) Câu trả lời chỉ có đoạn mô tả bảng bằng text không có thêm phần giải thích, hoặc ký hiệu lạ,.."},
]
pipe(messages)

Device set to use cuda:0


NameError: name 'tables' is not defined

In [ ]:
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings  # ho?c HuggingFaceEmbeddings
from langchain.schema import Document
from langchain_community.embeddings import HuggingFaceEmbeddings

# -----------------------
# 1?? Chu?n b? d? li?u
# -----------------------

# Gi? s? b?n ?ã có danh sách các Document:
# documents = [Document(page_content=text, metadata=metadata), ...]

# Ví d? nh?:
documents = [
    Document(
        page_content="Summary: ?o?n code tính t?ng hai s?.",
        metadata={"source": "file1.py", "type": "code", "full_text": "def add(a,b): return a+b"}
    ),
    Document(
        page_content="Summary: b?ng th?ng kê doanh thu n?m 2024.",
        metadata={"source": "report.xlsx", "type": "table", "full_text": "| Year | Revenue |\n| 2024 | 1.2M |"}
    ),
]

# -----------------------
# 2?? T?o embedding & l?u vào FAISS
# -----------------------
EMBED_MODEL = "keepitreal/vietnamese-sbert"
embedding_model = HuggingFaceEmbeddings(model_name=EMBED_MODEL) # ho?c HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vector_db = FAISS.from_documents(documents, embedding_model)

# -----------------------
# 3?? Truy v?n semantic
# -----------------------

query = "doanh thu n?m 2024"

# top_k = s? l??ng context l?y ra
results = vector_db.similarity_search(query, k=3)

# -----------------------
# 4?? X? lý k?t qu? tr? v?
# -----------------------

contexts = []
for doc in results:
    # L?y text g?c t? metadata n?u có, fallback v? page_content
    text_content = doc.metadata.get("full_text", doc.page_content)
    contexts.append({
        "text": text_content,
        "metadata": doc.metadata
    })

# -----------------------
# 5?? In ra context ?? dùng cho LLM
# -----------------------
for i, ctx in enumerate(contexts, start=1):
    print(f"--- Context {i} ---")
    print(f"Source: {ctx['metadata'].get('source', 'unknown')}")
    print(ctx["text"])
    print()


In [1]:
from langchain.docstore.document import Document
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings  # Ho?c HuggingFaceEmbeddings
import os

# -----------------------
# 1?? Chu?n b? d? li?u
# -----------------------
EMBED_MODEL = "keepitreal/vietnamese-sbert"
# Ví d? d? li?u: m?i Document có summary trong page_content,
# và n?i dung g?c (code, b?ng, context) trong metadata.
documents = [
    Document(
        page_content="Summary: ?o?n code tính t?ng hai s?.",
        metadata={
            "source": "file1.py",
            "type": "code",
            "full_text": "def add(a, b):\n    return a + b"
        }
    ),
    Document(
        page_content="Summary: b?ng th?ng kê doanh thu n?m 2024.",
        metadata={
            "source": "report.xlsx",
            "type": "table",
            "full_text": "| Year | Revenue |\n| 2024 | 1.2M |"
        }
    ),
    Document(
        page_content="Summary: mô t? chi?n l??c marketing quý I.",
        metadata={
            "source": "plan.txt",
            "type": "text",
            "full_text": "Chi?n l??c marketing t?p trung vào kênh social media và qu?ng cáo tr? phí."
        }
    )
]

# -----------------------
# 2?? T?o embedding model
# -----------------------


from langchain_community.embeddings import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings(model_name=EMBED_MODEL)


/tmp/ipykernel_3030273/2602696121.py:48: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name=EMBED_MODEL)
/opt/miniconda3/envs/agent/lib/python3.10/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
/opt/miniconda3/envs/agent/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/

In [4]:


# -----------------------
# 3?? T?o (ho?c load) ChromaDB
# -----------------------

persist_dir = "./chroma_db_vn"  # th? m?c l?u d? li?u

# vector_db = Chroma.from_documents(
#     documents=documents,
#     embedding=embedding,
#     persist_directory=persist_dir  # ?? có th? load l?i sau
# )

# # L?u vào ? ??a
# vector_db.persist()

vector_db = Chroma(
    persist_directory=persist_dir,
    collection_name="vn_chunks",
    embedding_function=embedding
)

# -----------------------
# 4?? Truy v?n semantic similarity
# -----------------------

query = " phát hiện đối tượng bằng mô hình YOLOv5s trên nhiều nguồn dữ liệu khác nhau bao gồm webcam, ảnh cục bộ, video, màn hình, thư mục ảnh, tệp văn bản liệt kê đường dẫn ảnh hoặc luồng phát, cũng như các link YouTube, luồng RTSP/RTMP/HTTP"

# Tìm top_k = 3 k?t qu? g?n nh?t
results = vector_db.similarity_search(query, k=3)
# print(results)

# -----------------------
# 5?? L?y context + metadata
# -----------------------

contexts = []
for doc in results:
    # print("----------------")
    # print(doc.metadata)
    # print(doc.page_content)
    text_content = doc.metadata.get("full_text", doc.page_content)
    contexts.append({
        "text": text_content,
        "metadata": doc.metadata
    })

# -----------------------
# 6?? In ra k?t qu? context
# -----------------------

# for i, ctx in enumerate(contexts, start=1):
#     print(f"--- Context {i} ---")
#     print(f"Source: {ctx['metadata'].get('type', 'unknown')}")
#     print(ctx["text"])
#     print()

In [6]:
print(vector_db.get()["metadatas"])

[{'Header 1': 'Public_003', 'document_file': '/data/AIRACE/example_folder/main.md'}, {'document_file': '/data/AIRACE/example_folder/main.md', 'Header 1': 'Public_003'}, {'Header 1': 'Public_003', 'document_file': '/data/AIRACE/example_folder/main.md'}, {'Header 1': 'Public_003', 'document_file': '/data/AIRACE/example_folder/main.md'}, {'document_file': '/data/AIRACE/example_folder/main.md', 'Header 1': 'Public_003'}, {'document_file': '/data/AIRACE/example_folder/main.md', 'Header 2': 'GIỚI THIỆU', 'Header 1': 'Nội dung chính'}, {'Header 2': 'GIỚI THIỆU', 'Header 1': 'Nội dung chính', 'document_file': '/data/AIRACE/example_folder/main.md'}, {'Header 2': 'GIỚI THIỆU', 'document_file': '/data/AIRACE/example_folder/main.md', 'Header 1': 'Nội dung chính'}, {'document_file': '/data/AIRACE/example_folder/main.md', 'Header 1': 'Nội dung chính', 'Header 2': 'GIỚI THIỆU'}, {'document_file': '/data/AIRACE/example_folder/main.md', 'Header 2': 'GIỚI THIỆU', 'Header 1': 'Nội dung chính'}, {'Header 

In [8]:
a = {"a":1, "b":2}
a.update({"a":2, "b":2})
print(a)

{'a': 2, 'b': 2}
